In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/processed-train-dataset/final_train_dataset.csv
/kaggle/input/train-labels/train_labels.csv
/kaggle/input/metadata/metadata.csv
/kaggle/input/processed-test-dataset/final_test_dataset.csv
/kaggle/input/submission-format-mars-spectrometry/submission_format.csv
/kaggle/input/validation-labels/val_labels.csv
/kaggle/input/supplemental-data/supplemental_metadata.csv


**Laying down the fundamental structure to proceed !** 

* step 1 : Adding val_features that is train[809:] to testing set keeping the labels unchanged same.
* step 2 : Using XGboost 

In [2]:
suppl_metadata = pd.read_csv("/kaggle/input/supplemental-data/supplemental_metadata.csv")

In [3]:
suppl_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1841 entries, 0 to 1840
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   sample_id           1841 non-null   object 
 1   split               1841 non-null   object 
 2   mars1_ega_split     1841 non-null   object 
 3   instrument_type     1841 non-null   object 
 4   basalt              1494 non-null   float64
 5   carbonate           1494 non-null   float64
 6   chloride            1494 non-null   float64
 7   iron_oxide          1494 non-null   float64
 8   oxalate             1494 non-null   float64
 9   oxychlorine         1494 non-null   float64
 10  phyllosilicate      1494 non-null   float64
 11  silicate            1494 non-null   float64
 12  sulfate             1494 non-null   float64
 13  sulfide             1494 non-null   float64
 14  carrier_gas         1841 non-null   object 
 15  different_pressure  1841 non-null   int64  
 16  featur

In [10]:
suppl_metadata[1490:1583]

,sample_id,split,mars1_ega_split,instrument_type,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,sulfide,carrier_gas,different_pressure,features_path,features_md5_hash
1490,EGA-S1502,supplemental,test,commercial,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,he,0,supplemental_features/EGA-S1502.csv,59402e65b5689766fbc18943c6c0a881
1491,EGA-S1503,supplemental,test,commercial,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,he,0,supplemental_features/EGA-S1503.csv,dd42287ddc74a8639495fa035ecc59a0
1492,EGA-S1504,supplemental,test,commercial,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,he,0,supplemental_features/EGA-S1504.csv,3030319eb6c5ed1cc6c15f8210991572
1493,EGA-S1505,supplemental,test,commercial,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,he,0,supplemental_features/EGA-S1505.csv,9edd10d43e50be65d21705d343b6e965
1494,EGA-X0000,supplemental,supplemental,commercial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,he,0,supplemental_features/EGA-X0000.csv,415d9be2aaa9151551ec2ce94aac3236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1578,EGA-X0084,supplemental,supplemental,commercial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,he,0,supplemental_features/EGA-X0084.csv,7990533f570f87834e3ed25fb4e0ec57
1579,EGA-X0085,supplemental,supplemental,commercial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,he,0,supplemental_features/EGA-X0085.csv,50d55aee3a0d9c267820e4292c1568d4
1580,EGA-X0086,supplemental,supplemental,commercial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,he,0,supplemental_features/EGA-X0086.csv,303f8cfd28874f341c1469bf7806fcd9
1581,EGA-X0087,supplemental,supplemental,commercial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,he,0,supplemental_features/EGA-X0087.csv,00d816808f5b1d966217f080e8812439


In [30]:
features = pd.read_csv("/kaggle/input/processed-train-dataset/final_train_dataset.csv")
labels = pd.read_csv("/kaggle/input/train-labels/train_labels.csv")
metadata = pd.read_csv("/kaggle/input/metadata/metadata.csv")
val_labels = pd.read_csv("/kaggle/input/validation-labels/val_labels.csv")
test = pd.read_csv("/kaggle/input/processed-test-dataset/final_test_dataset.csv")

In [31]:
## concating val features with testing dataset
final_test = pd.concat([features[809:], test])

In [32]:
## concating val set labels with our train set labels
final_labels = pd.concat([labels,val_labels])

In [33]:
final_labels.drop(['sample_id'], axis=1, inplace=True)

In [34]:
## adding metadata "derivatized" binary column to our features dataset
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1584 entries, 0 to 1583
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sample_id          1584 non-null   object 
 1   split              1584 non-null   object 
 2   derivatized        561 non-null    float64
 3   features_path      1584 non-null   object 
 4   features_md5_hash  1584 non-null   object 
dtypes: float64(1), object(4)
memory usage: 62.0+ KB


In [35]:
## now here is the problem for derivatized we only have 561 data points out of 1584 that is one third.

In [36]:
## so now our for features we will consider first 1120 values of derivatized and for testing we will have derivatized from [809:]
features['derivatized'] = metadata['derivatized'][0:1120]
final_test['derivatized'] = metadata['derivatized'][809:]

In [37]:
final_test['derivatized'] = final_test['derivatized'].fillna(value=0.0)

In [38]:
features['derivatized'] = features['derivatized'].fillna(value=0.0)
features.tail()

,Unnamed: 0,time,mass,intensity,derivatized
1116,1116,2.834383,74.970963,1.285254e+09,1.0
1117,1117,7.308333,146.799255,1.884232e+09,0.0
1118,1118,6.298967,221.081085,1.225335e+07,0.0
1119,1119,4.182017,133.996460,8.446767e+08,1.0
1120,1120,16.560683,221.229858,8.818310e+07,0.0


In [39]:
features.drop(['Unnamed: 0'], axis=1, inplace=True)
final_test.drop(['Unnamed: 0'], axis=1, inplace=True)

In [40]:
features.tail()

,time,mass,intensity,derivatized
1116,2.834383,74.970963,1.285254e+09,1.0
1117,7.308333,146.799255,1.884232e+09,0.0
1118,6.298967,221.081085,1.225335e+07,0.0
1119,4.182017,133.996460,8.446767e+08,1.0
1120,16.560683,221.229858,8.818310e+07,0.0


In [41]:
final_test['mass'] = np.log10(final_test['mass'])
final_test['intensity'] = np.log10(final_test['intensity'])
final_test['time'] = np.log10(final_test['time'])
features['mass'] = np.log10(features['mass'])
features['intensity'] = np.log10(features['intensity'])
features['time'] = np.log10(features['time'])

In [42]:
labels.drop(['sample_id'], axis=1, inplace=True)

In [43]:
## this step is optional i am just trying to remove time, i included it only because i thought it might be a proxy for temprature
#features.drop(['time'], axis=1, inplace=True)

In [44]:
final_labels.shape[0]

1121

In [45]:
for label in final_labels.columns:
    zero_values = len(final_labels[final_labels[f'{label}']== 0])
    one_values = len(final_labels[final_labels[f'{label}'] == 1])
    print(f"{label} zero values is {zero_values} and one value is {one_values}")
    print(f"{label} ratio is {np.round((zero_values/one_values),2)}")

aromatic zero values is 1018 and one value is 103
aromatic ratio is 9.88
hydrocarbon zero values is 814 and one value is 307
hydrocarbon ratio is 2.65
carboxylic_acid zero values is 956 and one value is 165
carboxylic_acid ratio is 5.79
nitrogen_bearing_compound zero values is 982 and one value is 139
nitrogen_bearing_compound ratio is 7.06
chlorine_bearing_compound zero values is 1095 and one value is 26
chlorine_bearing_compound ratio is 42.12
sulfur_bearing_compound zero values is 1085 and one value is 36
sulfur_bearing_compound ratio is 30.14
alcohol zero values is 1087 and one value is 34
alcohol ratio is 31.97
other_oxygen_bearing_compound zero values is 1095 and one value is 26
other_oxygen_bearing_compound ratio is 42.12
mineral zero values is 941 and one value is 180
mineral ratio is 5.23


In [46]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import log_loss
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

In [134]:
X_train, X_test, y_train, y_test = train_test_split(features, final_labels, test_size=0.17, random_state=2000)

In [135]:
classifier = MultiOutputClassifier(XGBClassifier(tree_method='gpu_hist'))

In [136]:
clf = Pipeline([
                ('classify', classifier)
               ])

params = {#'classify__estimator__colsample_bytree': 0.50,
          #'classify__estimator__gamma': 0.25,
          #'classify__estimator__scale_pos_weight': 1.05,
          #'classify__estimator__learning_rate': 0.01,
          #'classify__estimator__max_delta_step': 1.50,
          'classify__estimator__max_depth': 15,
          'classify__estimator__min_child_weight': 1,
          'classify__estimator__n_estimators': 248,
          'classify__estimator__eval_metric': 'logloss',
          'classify__estimator__colsample_bynode': 0.35,
          'classify__estimator__colsample_bylevel': 0.40,
          'classify__estimator__random_state': 2000,
          #'classify__estimator__subsample': 0.90,
          #'classify__estimator__booster': 'dart'
         }

_ = clf.set_params(**params)

In [137]:
clf.fit(X_train, y_train)

Pipeline(steps=[('classify',
                 MultiOutputClassifier(estimator=XGBClassifier(base_score=None,
                                                               booster=None,
                                                               callbacks=None,
                                                               colsample_bylevel=0.4,
                                                               colsample_bynode=0.35,
                                                               colsample_bytree=None,
                                                               early_stopping_rounds=None,
                                                               enable_categorical=False,
                                                               eval_metric='logloss',
                                                               gamma=None,
                                                               gpu_id=None,
                                                            

In [138]:
val_preds = clf.predict_proba(X_test)

In [139]:
val_preds = np.array(val_preds)[:,:,1].T
#val_preds

In [140]:
loss = log_loss(np.ravel(y_test), np.ravel(val_preds))
loss

0.14461251716449303

In [91]:
## perdicting model out of this shit 

final_predictions = clf.predict_proba(final_test)

In [92]:
final_predictions = np.array(final_predictions)[:,:,1].T

In [93]:
submission = pd.read_csv("/kaggle/input/submission-format-mars-spectrometry/submission_format.csv")

In [94]:
print(final_predictions.shape[0]-1)

774


In [95]:
sample_id = submission['sample_id']

In [96]:
submission.drop(['sample_id'],axis=1, inplace=True)

In [97]:
submission.iloc[0] = final_predictions[0]
submission.head()

,aromatic,hydrocarbon,carboxylic_acid,nitrogen_bearing_compound,chlorine_bearing_compound,sulfur_bearing_compound,alcohol,other_oxygen_bearing_compound,mineral
0,0.251334,0.23938,0.133166,0.41519,0.270016,0.286417,0.257461,0.24802,0.747394
1,0.500000,0.50000,0.500000,0.50000,0.500000,0.500000,0.500000,0.50000,0.500000
2,0.500000,0.50000,0.500000,0.50000,0.500000,0.500000,0.500000,0.50000,0.500000
3,0.500000,0.50000,0.500000,0.50000,0.500000,0.500000,0.500000,0.50000,0.500000
4,0.500000,0.50000,0.500000,0.50000,0.500000,0.500000,0.500000,0.50000,0.500000


In [98]:
final_predictions.shape

(775, 9)

In [99]:
for i in range(0,final_predictions.shape[0],1):
    submission.iloc[i] = final_predictions[i]

In [100]:
submission.tail()

,aromatic,hydrocarbon,carboxylic_acid,nitrogen_bearing_compound,chlorine_bearing_compound,sulfur_bearing_compound,alcohol,other_oxygen_bearing_compound,mineral
770,0.012327,0.135387,0.161384,0.004805,0.000403,0.015769,0.016035,0.003903,0.012377
771,0.003814,0.011582,0.213748,0.017218,0.000589,0.001708,0.007309,0.000917,0.403072
772,0.024499,0.529726,0.003117,0.012043,0.000208,0.027529,0.013178,0.007550,0.035308
773,0.029195,0.059907,0.301858,0.016964,0.000237,0.005992,0.010180,0.001706,0.476181
774,0.829299,0.118911,0.024267,0.011490,0.000248,0.005148,0.044446,0.001166,0.704945


In [101]:
submission['sample_id'] = sample_id

In [102]:
submission.head()

,aromatic,hydrocarbon,carboxylic_acid,nitrogen_bearing_compound,chlorine_bearing_compound,sulfur_bearing_compound,alcohol,other_oxygen_bearing_compound,mineral,sample_id
0,0.251334,0.239380,0.133166,0.415190,0.270016,0.286417,0.257461,0.248020,0.747394,S0809
1,0.039030,0.106330,0.001332,0.006414,0.000203,0.007129,0.006729,0.001166,0.975087,S0810
2,0.008871,0.471804,0.009785,0.024856,0.003079,0.046762,0.009173,0.020568,0.613093,S0811
3,0.078670,0.075311,0.000748,0.031584,0.001569,0.003835,0.006356,0.010940,0.784786,S0812
4,0.136115,0.024828,0.001882,0.007149,0.000203,0.007129,0.006729,0.001166,0.979270,S0813


In [103]:
submission.to_csv("/kaggle/working/submission15.csv")

In [141]:
X = np.array(features)
y = np.array(final_labels)
oof_preds = np.zeros(y.shape)
test_preds = np.zeros((final_test.shape[0],y.shape[1]))
oof_losses = []
NFOLDS = 15
kf = KFold(n_splits=NFOLDS)
for fn, (trn_idx, val_idx) in enumerate(kf.split(X, y)):
    print('Starting fold: ', fn)
    X_train, X_val = X[trn_idx], X[val_idx]
    y_train, y_val = y[trn_idx], y[val_idx]
    
    # drop where cp_type==ctl_vehicle (baseline)
    #ctl_mask = X_train[:,0]=='ctl_vehicle'
    #X_train = X_train[~ctl_mask,:]
    #y_train = y_train[~ctl_mask]
    
    clf.fit(X_train, y_train)
    val_preds = clf.predict_proba(X_val) # list of preds per class
    val_preds = np.array(val_preds)[:,:,1].T # take the positive class
    oof_preds[val_idx] = val_preds
    
    loss = log_loss(np.ravel(y_val), np.ravel(val_preds))
    oof_losses.append(loss)
    preds = clf.predict_proba(final_test)
    preds = np.array(preds)[:,:,1].T # take the positive 
    print(preds.shape)
    test_preds += preds / NFOLDS
    
print(oof_losses)
print('Mean OOF loss across folds', np.mean(oof_losses))
print('STD OOF loss across folds', np.std(oof_losses))

Starting fold:  0
(775, 9)
Starting fold:  1
(775, 9)
Starting fold:  2
(775, 9)
Starting fold:  3
(775, 9)
Starting fold:  4
(775, 9)
Starting fold:  5
(775, 9)
Starting fold:  6
(775, 9)
Starting fold:  7
(775, 9)
Starting fold:  8
(775, 9)
Starting fold:  9
(775, 9)
Starting fold:  10
(775, 9)
Starting fold:  11
(775, 9)
Starting fold:  12
(775, 9)
Starting fold:  13
(775, 9)
Starting fold:  14
(775, 9)
[0.1726214623398792, 0.16016578525873937, 0.2380729092803732, 0.2155602991783181, 0.14466241104794553, 0.14615669826039165, 0.12775934171591263, 0.2258102571763771, 0.1575335650055031, 0.1682500108281642, 0.1489847066318422, 0.20947545012510288, 0.169965096473742, 0.17604192425937368, 0.16779394457736382]
Mean OOF loss across folds 0.17525692414393523
STD OOF loss across folds 0.03131354585566276
